### WORKING CODE with backtest - individiual tickers - mini

In [1]:
import sqlite3
import pandas as pd
import logging
import backtrader as bt
import quantstats as qs
import os
from bt_strategy import (
    evaluate_strategies,
    buy_and_hold_strategy,
    close_vs_sma_strategy,
    mean_reversion_strategy,
    prev_peak_strategy,
    random_strategy,
    prev_peak_nodrop_strategy,
    close_vs_sma_nodrop_strategy,
    mean_reversion_nodrop_strategy,
    anti_drop_strategy
)
import warnings
warnings.filterwarnings("ignore")

# Set Pandas option to display all columns
pd.set_option('display.max_columns', None)

# Evaluate strategies with multiple trials and parallel processing
n_trials = 10  # Number of trials per strategy per ticker
n_jobs = -1     # Use all available CPU cores

# Set up logging configuration
logging.basicConfig(level=logging.INFO)

# Define your reporting folder (ensure this exists)
REPORTS_FOLDER = "QuantStats_Reports"
os.makedirs(REPORTS_FOLDER, exist_ok=True)  # Create the folder if it doesn't exist

# Constants
DB_PATH = r"/Users/mukeshwaranbaskaran/Documents/MFE Notes/Term 4/AFP/Code/Download_This_Folder/1_financial_data.db"
START_DATE = '2020-01-01'
END_DATE = '2023-12-31'

# Function to load distinct tickers from the SQLite database
def load_distinct_tickers_from_db():
    """Load distinct tickers from the database within the specified date range."""
    conn = sqlite3.connect(DB_PATH)
    query = """
        SELECT DISTINCT ticker
        FROM merged_data
        WHERE date BETWEEN ? AND ?
    """
    distinct_tickers = pd.read_sql(query, conn, params=(START_DATE, END_DATE))
    conn.close()
    return distinct_tickers

# Load tickers and order alphabetically
tickers = load_distinct_tickers_from_db()

# Assuming the result is a DataFrame, extract the 'ticker' column
tickers_list_full = tickers['ticker'].tolist()
tickers_list_mini = ['REGN','UTHR','CLDX','XOMA','AXGN','HRTX','OCUP','CAPR']#,'OMER','IRWD','CRMD']

# Function to load data from the SQLite database
def load_data_from_db(tickers_list):
    """Load data from the database within the specified date range, excluding tickers with no data."""
    conn = sqlite3.connect(DB_PATH)
    
    # Convert tickers_list into a format suitable for the SQL IN clause (comma-separated string)
    tickers_tuple = tuple(tickers_list)
    
    # Make sure tickers_tuple is not empty to prevent SQL errors
    if not tickers_tuple:
        return pd.DataFrame()  # Return an empty DataFrame if no tickers are provided

    query = """
        SELECT *
        FROM merged_data
        WHERE date BETWEEN ? AND ?
        AND ticker IN ({})
    """.format(','.join(['?'] * len(tickers_tuple)))  # Dynamically insert placeholders for each ticker

    # Run the query with the tickers_list as parameters
    params = (START_DATE, END_DATE) + tickers_tuple
    merged_data = pd.read_sql(query, conn, params=params)
    
    conn.close()
    return merged_data

# Load and preprocess data
merged_data = load_data_from_db(tickers_list_mini)
merged_data['date'] = pd.to_datetime(merged_data['date'], errors='coerce')
merged_data.sort_values('date', inplace=True)
merged_data.set_index('date', inplace=True)

equities_cols_needed = ['permno', 'cusip', 'bid', 'ask', 'vol', 'shrout', 'prc', 'mktcap', 'ticker']
merged_data_equities = merged_data[equities_cols_needed]

# Drop duplicate rows & reset index
merged_data_equities_no_duplicates = merged_data_equities.drop_duplicates()
merged_data_equities_no_duplicates_reset_index = merged_data_equities_no_duplicates.reset_index()

# ERRORS WITH LESS THAN 800 rows of data per ticker
def filter_tickers_with_min_rows(df, min_rows=800):
    # Group by 'ticker' and count the rows for each ticker
    ticker_counts = df.groupby('ticker').size()
    
    # Identify tickers with more than the minimum required rows
    valid_tickers = ticker_counts[ticker_counts >= min_rows].index.tolist()
    
    # Filter the DataFrame to only include rows for these tickers
    filtered_data_800 = df[df['ticker'].isin(valid_tickers)]
    
    #print(f"Filtered DataFrame with only tickers that have at least {min_rows} rows.")
    
    return filtered_data_800

filtered_data_800 = filter_tickers_with_min_rows(merged_data_equities_no_duplicates_reset_index)

# Convert to OHLC format for backtrader
def convert_to_ohlc_format(df):
    df['date'] = pd.to_datetime(df['date'])
    grouped = df.groupby(['date', 'ticker'])
    result = grouped.agg(
        Open=('bid', 'first'),
        High=('ask', 'max'),
        Low=('bid', 'min'),
        Close=('prc', 'last'),  # Use 'prc' for closing price here
        Adj_Close=('prc', 'last'),
        Volume=('vol', 'sum')
    ).reset_index()
    result.set_index('date', inplace=True)
    result.rename(columns={'ticker': 'Ticker'}, inplace=True)
    return result

merged_equities_OHLC = convert_to_ohlc_format(filtered_data_800)

# BACKTESTING 

# Initialize Cerebro engine
cerebro = bt.Cerebro()

# Add data feeds for all tickers
unique_tickers = merged_equities_OHLC['Ticker'].unique()

for ticker in unique_tickers:
    print(f"Adding data for ticker: {ticker}")
    
    # Filter data for the current ticker
    ticker_data = merged_equities_OHLC[merged_equities_OHLC['Ticker'] == ticker]
    ticker_data.index = pd.to_datetime(ticker_data.index)
    ticker_data = ticker_data[['Open', 'High', 'Low', 'Close', 'Volume']]
    
    # Define custom PandasData feed for backtrader
    class PandasData(bt.feeds.PandasData):
        params = (
            ('datetime', None),
            ('open', 'Open'),
            ('high', 'High'),
            ('low', 'Low'),
            ('close', 'Close'),
            ('volume', 'Volume'),
            ('openinterest', None),
        )

    # Add this ticker's data feed to Cerebro
    data_feed = PandasData(dataname=ticker_data)
    cerebro.adddata(data_feed, name=ticker)

# Define strategies to evaluate
strategies = [
    buy_and_hold_strategy,
    close_vs_sma_strategy,
    mean_reversion_strategy,
    prev_peak_strategy,
    random_strategy,
    prev_peak_nodrop_strategy,
    close_vs_sma_nodrop_strategy,
    mean_reversion_nodrop_strategy,
    anti_drop_strategy
]

# Prepare logs for each ticker's OHLC data
logs = {ticker: merged_equities_OHLC[merged_equities_OHLC['Ticker'] == ticker] for ticker in unique_tickers}

results_df = evaluate_strategies(strategies, logs, n_trials, n_jobs)

# Save results to a CSV file (optional)
results_df.to_csv(f"{REPORTS_FOLDER}/strategy_evaluation_results.csv", index=False)

# Function to generate QuantStats reports
def generate_quantstats_report(logs, strategies):
    """Generate and save one QuantStats report per strategy."""
    if not logs:
        logging.warning("No logs available to generate reports.")
        return

    for strategy in strategies:
        try:
            combined_data = pd.DataFrame()  # To store combined data for each strategy
            for ticker in logs.keys():  # Use only the tickers in 'logs'
                ticker_data = logs[ticker]
                
                # Ensure the ticker data is in the format QuantStats expects
                # 'Close' as the column representing the stock's price (closing price)
                combined_data[ticker] = ticker_data['Close']

            # Create a QuantStats report for the combined data of this strategy
            qs.extend_pandas()  # Ensure QuantStats can extend Pandas functionalities
            report_filename = f'quantstats_report_{strategy.__name__}.html'
            report_path = os.path.join(REPORTS_FOLDER, report_filename)
            qs.reports.html(combined_data, output=report_path, title=f'QuantStats Report for {strategy.__name__}')
            logging.info(f"QuantStats report generated for strategy: {strategy.__name__}")
            logging.info(f"Report saved to: {report_path}")

        except Exception as e:
            logging.error(f"Error generating QuantStats report for strategy {strategy.__name__}: {e}")

# Generate the QuantStats reports for each strategy
generate_quantstats_report(logs, strategies)

# EVALUATION - Strategy performance stats
# Grouping results by strategy name to calculate mean value and max drawdown
strategy_summary = results_df.groupby('strategy').agg(
    mean_value=('value', 'mean'),
    max_drawdown=('dropdown', 'max')
)

# Print the evaluation summary
logging.info("Strategy Evaluation Summary:")
logging.info(strategy_summary)

Adding data for ticker: HRTX
Adding data for ticker: REGN
Adding data for ticker: UTHR
Adding data for ticker: CLDX


100%|██████████| 360/360 [00:08<00:00, 44.74it/s]
INFO:root:QuantStats report generated for strategy: buy_and_hold_strategy
INFO:root:Report saved to: QuantStats_Reports/quantstats_report_buy_and_hold_strategy.html
INFO:root:QuantStats report generated for strategy: close_vs_sma_strategy
INFO:root:Report saved to: QuantStats_Reports/quantstats_report_close_vs_sma_strategy.html
INFO:root:QuantStats report generated for strategy: mean_reversion_strategy
INFO:root:Report saved to: QuantStats_Reports/quantstats_report_mean_reversion_strategy.html
INFO:root:QuantStats report generated for strategy: prev_peak_strategy
INFO:root:Report saved to: QuantStats_Reports/quantstats_report_prev_peak_strategy.html
INFO:root:QuantStats report generated for strategy: random_strategy
INFO:root:Report saved to: QuantStats_Reports/quantstats_report_random_strategy.html
INFO:root:QuantStats report generated for strategy: prev_peak_nodrop_strategy
INFO:root:Report saved to: QuantStats_Reports/quantstats_repo

### If mean_value = 1.1293, it means the strategy has generated a 12.93% gain. ; max_drawdown is percentage units